In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import vgg16
from torchvision import datasets
from torchvision.transforms import ToTensor, Compose, Resize
from torch.utils.data import DataLoader
torch.manual_seed(0)

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
epochs = 3
batch_size = 128

In [4]:
data = datasets.ImageFolder(root = 'data',
                           transform = Compose(
                               [
                                   Resize((224, 224)),
                                   ToTensor()
                               ]
                           )
)

In [5]:
n_classes = len(data.classes)

In [6]:
data_size = len(data)
train_perc = 0.6
val_perc = test_perc = 0.2
train_set, val_set, test_set = torch.utils.data.random_split(data,
 [
     int(data_size * train_perc),
     int(data_size * val_perc),
     data_size - int(data_size * train_perc) - int(data_size * val_perc),
  ]
                                                             )
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True)

In [7]:
vgg = vgg16(weights="VGG16_Weights.IMAGENET1K_V1").to(device)

In [8]:
vgg.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [9]:
vgg.classifier[6].out_features = n_classes

In [10]:
vgg.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=5, bias=True)
)

In [11]:
criterion = nn.CrossEntropyLoss()

In [12]:
optimizer = optim.AdamW(vgg.parameters(), lr=0.001)

In [13]:
def train(model, criterion, optimizer, num_epochs=25):
    train_losses_per_epoch = []
    val_losses_per_epoch = []
    model = model.to(device)
    
    for epoch in range(num_epochs):
        train_losses_per_batch = []
        val_losses_per_batch = []
    
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)
            pred = model(images)
            loss = criterion(pred, labels)
            loss.backward()
            optimizer.step()
            train_losses_per_batch.append(loss)
        
        
        with torch.no_grad():
        
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device)
                pred = model(images)
                loss = criterion(pred, labels)
                val_losses_per_batch.append(loss)
        
        train_losses_per_epoch.append(torch.Tensor(train_losses_per_batch).mean())
        val_losses_per_epoch.append(torch.Tensor(val_losses_per_batch).mean())
        print(f"{epoch + 1}: train_loss: {train_losses_per_epoch[-1]} val_loss: {val_losses_per_epoch[-1]}")
    
    return (
        train_losses_per_epoch,
        val_losses_per_epoch
    )

In [14]:
train_loss, val_loss = train(vgg, criterion, optimizer, epochs)

1: train_loss: 8.025720596313477 val_loss: 6.761537075042725
2: train_loss: 46.70078659057617 val_loss: 40.78505325317383
3: train_loss: 19.90152931213379 val_loss: 5.845839500427246


In [15]:
torch.save(vgg, "./models/vgg_flower_trained.pt")